Linked Structures
=================



## Agenda



1.  Motives
2.  Objectives
3.  Mechanisms
4.  Linked Data Structures



## 1.  Motives



In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
from timeit import timeit

def time_array_front_insert_delete(n):
    return timeit('lst.insert(0, None) ; del lst[0]',
                  'lst = list(range({}))'.format(n),
                  number=1000)

ns = np.linspace(100, 10000, 50)
plt.plot(ns, [time_array_front_insert_delete(int(n)) for n in ns], 'ro')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [17]:
# consider:

def concatenate(arr1, arr2):
    """Concatenates the contents of arr1 and arr2 as efficiently (time-wise)
    as possible, so that the resulting structure can be used to index all
    combined elements (arr1's followed by arr2's)."""

    # option 1: O(?)
    for x in arr2:
        arr1.append(x)
    return arr1

def concatenate2(arr1, arr2):
    # option 2: O(?)
    arr1.extend(arr2)
    return arr1

def concatenate3(arr1, arr2):
    # option 3: O(?)
    return arr1 + arr2

## 1.  Objectives



We would like a new data storage mechanism for constructing data
structures that:

-   does not require monolithic, contiguous memory allocation,
-   allows individual elements to be flexibly and efficiently reorganized,
-   and preserves the ability to locate (e.g., via position) and iterate

over elements



## 1.  Mechanisms



### 3.1. Two-Element Lists



In [57]:
#data items
i1 = 'lions'
i2 = 'tigers'
i3 = 'bears'
i4 = 'oh, my'

In [58]:
# creating individual "links"
l4 = [i4, None]
l3 = [i3, l4]
l2 = [i2, l3]
l1 = [i1, l2]
l1
#l1 = [i1, [i2, [i3, [i4, None]]]]

['lions', ['tigers', ['bears', ['oh, my', None]]]]

In [59]:
# link-ing them together


In [60]:
# iteration
def printlist(head):
    cur = head
    while cur != None:
        print(cur[0])
        cur = cur[1] 
printlist(l1)

lions
tigers
bears
oh, my


In [61]:
# prepending

i0 = 'walruses'
l0 = [i0, l1]
head = l0
printlist(head)

walruses
lions
tigers
bears
oh, my


In [62]:
def prepend_list(head, el):
    newcell = [el, head] #1
    return newcell       #1

newhead = prepend_list(l1, 'walruses')
printlist(newhead)

walruses
lions
tigers
bears
oh, my


In [63]:
def get_linked_list(head, pos): #O(n)
    cur = head                  #1
    for i in range(0, pos):     #O(n)
        cur = cur[1]            #O(n)
        if not cur:             #O(n)
            raise IndexError() #1
    return cur[0]               #1


get_linked_list(newhead, 3)

SyntaxError: invalid syntax (<ipython-input-63-cc0d70a5174c>, line 6)

In [55]:
# insertion
def insert_linked_list(head, pos, el):
    cur = head
    for i in range(0, pos):
        cur = cur[1]
        if not cur:
            raise IndexError()
    newel = [el, cur[1]]
    cur[1] = newel
    
i2_5 = 'elephants'
for i in range(0, 4):
    insert_linked_list(l0, 2, i2_5) #did run it 4 times
printlist(head)

walruses
lions
tigers
elephants
elephants
elephants
elephants
bears
oh, my


In [ ]:
#find cell
def get_cell_at_pos(head, pos):
    cur = head
    for i in range(0, pos):
        cur = cur[1]
    return cur

In [56]:
# deletion
def delete_linked_list(head, pos):
    cur = get_cell_at_pos(head, pos-1)
    nxt = cur[1]
    cur[1] = nxt[1]

delete_linked_list(head, 3)
printlist(head)

NameError: name 'get_cell_at_pos' is not defined

### 3.2. "Link" objects



In [69]:
class Link:
    def __init__(self, val, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        nextrep = "None" if not self.next else self.next.__repr__()
        return f"link({self.val.__repr__(), nextrep})"

In [70]:
# manually constructing a list
head = Link('lions', Link('bear', Link('fleas', None)))
print(head)

link(("'lions'", 'link(("\'bear\'", \'link(("\\\'fleas\\\'", \\\'None\\\'))\'))'))


In [47]:
# prepending

def prepend(l, val):
    return Link(val,l)

In [48]:
l = None
for x in range(10):
    l = prepend(l, x)

In [49]:
# iterator

def link_iterator(l):
    cur = l
    while cur.next:
        yield cur.val
        cur = cur.next

In [50]:
for x in link_iterator(l):
    print(x)

None


In [71]:
# iteration based on a recursive pattern

def link_iterator_rec(l):
    if l:
        yield l.val
        for val in link_iterator_rec(l.next):
            yield val

In [52]:
for x in link_iterator_rec(l):
    print(x)

None


## 1.  Linked Data Structures



### 4.1 Linked List



In [73]:
class LinkedList:
    class Link:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next

    def __init__(self):
        self.head = None
        self.len = 0 #store length --> quicker!
    
    def __len__(self):  # O(n)
        return self.len
        #cur = self.head #1
        #length = 0
        #while cur:          
        #    cur = cur.next
        #    length += 1
        #return length
    
    def normalize_index(i):
        #add checks
        if i<0:
            i = len(self) + i
        return i
    
    def find_link(self, pos): #O(n)
        cur = self.head
        for i in range(0, pos):
            cur = cur.next
            if not cur:
                raise IndexError
 
    def __getitem__(self, index):
        return self.find_link(index).val
  
    def __setitem__(self, idx, val):
        cur = self.find_link(idx)
        cur.val = val

    def prepend(self, val):
        self.head = self.Link(val, self.head)
        self.len += 1
    
    def insert_linked_list(head, pos, val):
       npos = LinkedList.normalize_index(pos)
       assert(npos >= 0 and npos <= len(self))
       if pos == 0:
           newcell = self.Link(val, self.head)
           self.head = newcell
        else:
            link = self.find_link(pos - 1)
            newcell = self.Link(val, link.next)
            link.next = newcell

    def __delitem__(self, pos):
        npos = LinkedList.normalize_index(pos)
        assert(npos >= 0 and npos < len(self))
        if pos == 0
            self.head = self.head.next
        else:
            cur = self.find_link(pos-1)
            nxt = cur.next.next
        
    def __iter__(self):
        cur = self.head
        while cur:
            yield cur.val ##generator - object that implements iteration. Like "return" but does not stop execution of function. 
            cur = cur.next

    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [74]:
l = LinkedList()
for x in range(10):
    l.prepend(x)
l

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

### 4.2 Binary Tree



In [1]:
class BinaryLink:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [1]:
# manual construction of a "tree" representing the expression ((5+3)*(8-4))
t = BinaryLink('*')
t.left = BinaryLink('+')
t.left.left  = BinaryLink('5')
t.left.right = BinaryLink('3')
t.right = BinaryLink('-')
t.right.left  = BinaryLink('8')
t.right.right = BinaryLink('4')

In [1]:
def print_expr_tree(t):
    if t:
        if not t.val.isdigit():
            print('(', end='')
        print_expr_tree(t.left)
        print(t.val, end='')
        print_expr_tree(t.right)
        if not t.val.isdigit():
            print(')', end='')

In [1]:
print_expr_tree(t)

### 4.3 N-ary Tree



In [1]:
class NaryLink:
    def __init__(self, val, n=2):
        self.val = val
        self.children = [None] * n

    def __getitem__(self, idx):
        return self.children[idx]

    def __setitem__(self, idx, val):
        self.children[idx] = val

    def __iter__(self):
        for c in self.children:
            yield c

In [1]:
root = NaryLink('Kingdoms', n=5)

root[0] = NaryLink('Animalia', n=35)
root[1] = NaryLink('Plantae', n=12)
root[2] = NaryLink('Fungi', n=7)
root[3] = NaryLink('Protista', n=5)
root[4] = NaryLink('Monera', n=5)

root[2][0] = NaryLink('Chytridiomycota')
root[2][1] = NaryLink('Blastocladiomycota')
root[2][2] = NaryLink('Glomeromycota')
root[2][3] = NaryLink('Ascomycota')
root[2][4] = NaryLink('Basidiomycota')
root[2][5] = NaryLink('Microsporidia')
root[2][6] = NaryLink('Neocallimastigomycota')

def tree_iter(root):
    if root:
        yield root.val
        for c in root:
            yield from tree_iter(c)

In [1]:
for x in tree_iter(root):
    print(x)